In [6]:
import pandas as pd

import pyodbc
import sqlalchemy
import urllib

import pickle
import os.path
import google
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [7]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [8]:
# Google API Check Function
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [9]:
gsheet_api_check(SCOPES)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=47187434094-l8ua38hd32l2qqjq1ms6mm8boopf66er.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56690%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=VlihEie53kRez4gn2XZ8jzW9RN9P3S&access_type=offline


In [10]:
# Google Sheet Data Pull Function
def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

In [11]:
# Connect to MS SQL Server
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=winwin; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect(conn_string);
crs = conn.cursor()
conn.autocommit = True

In [12]:
# MS SQL Execute Query
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [13]:
# MS SQL Read Query
def read_query (conn, query):
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [14]:
# Create Schema
# query = 'CREATE SCHEMA Sales'
# execute_query(conn, query)

In [15]:
# Create Table
# query = """CREATE TABLE Sales.AllSales (OrderName NVARCHAR (50),
#  									  	OrderId NVARCHAR (50),
#  									  	InvoiceId NVARCHAR (50),
#  										CustomerName NVARCHAR (50),
#  										InvStatus NVARCHAR (50),
#  										Fulfillment NVARCHAR (50),
#  										Method1 NVARCHAR (50),
#  										Method2 NVARCHAR (50),
#  										Method3 NVARCHAR (50),
#  										Method4 NVARCHAR (50),
#  										PaySource NVARCHAR (50),
#  										Store NVARCHAR (50),
#  										CreateDate DATE,
#  										PayDate DATE,
#  										FulfillDate DATE,
#  										CancelDate DATE,
#  										CustomerId NVARCHAR (50),
#  										Phone NVARCHAR (50),
#  										Email NVARCHAR (50),
#  										Item NVARCHAR (MAX),
#  										Vendor NVARCHAR (50),
#  										SKU NVARCHAR (MAX),
#  										New_Item NVARCHAR (MAX),
#  										Shipped INT,
#  										DwUnitPrice NVARCHAR (50),
#  										Ordered INT,
#  										UpUnitPrice NVARCHAR (50),
#  										BillingName NVARCHAR (50),
#  										BillingStreet NVARCHAR (MAX),
#  										BillingCity NVARCHAR (50),
#  										BillingProvince NVARCHAR (50),
#  										BillingZip INT,
#  										ShippingName NVARCHAR (50),
#  										ShippingStreet NVARCHAR (MAX),
#  										ShippingCity NVARCHAR (50),
#  										ShippingProvince NVARCHAR (50),
#  										ShippingZip INT,
#  										ShippingMethod NVARCHAR (50),
#  										TaxManassas NVARCHAR (50),
#  										TaxVirginia NVARCHAR (50),
#  										TaxGoogle NVARCHAR (50),
#  										TaxNebraska NVARCHAR (50),
#  										TaxTennessee NVARCHAR (50),
#  										TaxNevada NVARCHAR (50),
#  										TaxNewYork NVARCHAR (50),
#  										TaxOther NVARCHAR (50),
#  										TaxTotal NVARCHAR (50),
#  										Subtotal NVARCHAR (50),
#  										Discount NVARCHAR (50),
#  										Shipping NVARCHAR (50),
#  										TotalExcTax NVARCHAR (50),
#  										InvoiceTotal NVARCHAR (50),
#  										PaidTotal NVARCHAR (50),
#  										AmountDue NVARCHAR (50),
#  										Refund NVARCHAR (50));"""

# execute_query(conn, query)

In [16]:
# Google Sheet Data Pull
SPREADSHEET_ID = '13yL-WmxrqPGZqHi7ba1MI_UcXqd1F-I1Jb4bNrMew8s'

#Pulls data from the entire spreadsheet tab.
#DATA_TO_PULL = 'AllSales'

#Pulls data only from the specified range of cells.
DATA_TO_PULL = 'AllSales!A1:AV10000'

data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
df = pd.DataFrame(data[1:], columns=data[0])
print(df.shape)
df.head(5)

COMPLETE: Data copied
(2072, 48)


,OrderName,OrderId,InvoiceId,Store,CustomerName,InvStatus,Fulfillment,Method1,Method2,Method3,...,ShippingMethod,TaxTotal,Subtotal,Discount,Shipping,TotalExcTax,InvoiceTotal,PaidTotal,AmountDue,Refund
0,#1002,1002,1002,,Osman Karabacak,Refunded,Fulfilled,Shopify,,,...,Up To 50 Miles,0.79,79.00,0.00,75.00,154.00,154.79,158.74,-3.95,158.74
1,#1003,1003,1003,,Ahmet Cetinkaya,Paid,Fulfilled,Shopify,,,...,Pickup From Store,8.39,"1,199.00",359.70,0.00,839.30,847.69,889.66,-41.97,0.00
2,#1004,1004,1004,,Corina Menjivar,Paid,Fulfilled,Cash,,,...,,6.15,615.00,0.00,0.00,615.00,621.15,651.90,-30.75,0.00
3,#1005,1005,1005,,No Customer Name,Paid,Fulfilled,Cash,,,...,,1.35,135.00,0.00,0.00,135.00,136.35,143.10,-6.75,0.00
4,#1006,1006,1006,,Zeynel Bal,Paid,Fulfilled,Paypal,,,...,Pickup From Store,4.99,499.00,0.00,0.00,499.00,503.99,528.94,-24.95,0.00


In [17]:
# Use winwin database
query = 'USE winwin'
execute_query(conn, query)

Query Succeessful!


In [18]:
# Truncate the table
query = 'TRUNCATE TABLE Sales.AllSales'
execute_query(conn, query)

NameError: name 'Error' is not defined

In [13]:
# Read the Empty Table
pd.read_sql("SELECT * FROM Sales.AllSales", con = conn)

,OrderName,OrderId,InvoiceId,CustomerName,InvStatus,Fulfillment,Method1,Method2,Method3,Method4,...,TaxOther,TaxTotal,Subtotal,Discount,Shipping,TotalExcTax,InvoiceTotal,PaidTotal,AmountDue,Refund


In [14]:
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():
     crs.execute(""" INSERT INTO Sales.AllSales (OrderName, OrderId, InvoiceId, CustomerName, InvStatus, Fulfillment,
                                             Method1, Method2, Method3, Method4, PaySource, Store, CreateDate,
                                             PayDate, FulfillDate, CancelDate, CustomerId, Phone, Email, Item,
                                             Vendor, SKU, New_Item, Shipped, DwUnitPrice, Ordered, UpUnitPrice,
                                             BillingName, BillingStreet, BillingCity, BillingProvince,
                                             BillingZip, ShippingName, ShippingStreet, ShippingCity,
                                             ShippingProvince, ShippingZip, ShippingMethod, TaxManassas,
                                             TaxVirginia, TaxGoogle, TaxNebraska, TaxTennessee, TaxNevada,
                                             TaxNewYork, TaxOther, TaxTotal, Subtotal, Discount, Shipping,
                                             TotalExcTax, InvoiceTotal, PaidTotal, AmountDue, Refund)
                              VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                      ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                      ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                      ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                      ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                      ?, ?, ?, ?, ?) """,
                 row.OrderName, row.OrderId, row.InvoiceId, row.CustomerName, row.InvStatus,
                 row.Fulfillment, row.Method1, row.Method2, row.Method3, row.Method4, row.PaySource,
                 row.Store, row.CreateDate, row.PayDate, row.FulfillDate, row.CancelDate, row.CustomerId,
                 row.Phone, row.Email, row.Item, row.Vendor, row.SKU, row.New_Item, row.Shipped,
                 row.DwUnitPrice, row.Ordered, row.UpUnitPrice, row.BillingName, row.BillingStreet,
                 row.BillingCity, row.BillingProvince, row.BillingZip, row.ShippingName, row.ShippingStreet,
                 row.ShippingCity, row.ShippingProvince, row.ShippingZip, row.ShippingMethod, row.TaxManassas,
                 row.TaxVirginia, row.TaxGoogle, row.TaxNebraska, row.TaxTennessee, row.TaxNevada,
                 row.TaxNewYork, row.TaxOther, row.TaxTotal, row.Subtotal, row.Discount, row.Shipping,
                 row.TotalExcTax, row.InvoiceTotal, row.PaidTotal, row.AmountDue, row.Refund)
conn.commit()
crs.close()

In [15]:
# Read the Full Table
df = pd.read_sql("SELECT * FROM Sales.AllSales", con = conn)
print(df.shape)
df.head(5)

(1309, 55)


,OrderName,OrderId,InvoiceId,CustomerName,InvStatus,Fulfillment,Method1,Method2,Method3,Method4,...,TaxOther,TaxTotal,Subtotal,Discount,Shipping,TotalExcTax,InvoiceTotal,PaidTotal,AmountDue,Refund
0,#1002,1002,1002,Osman Karabacak,refunded,fulfilled,Shopify,,,,...,0.00,0.79,79.00,0.00,75.00,154.00,154.79,158.74,-3.95,158.74
1,#1003,1003,1003,Ahmet Cetinkaya,paid,fulfilled,Shopify,,,,...,0.00,8.39,"1,199.00",359.70,0.00,839.30,847.69,889.66,-41.97,0.00
2,#1004,1004,1004,Corina Menjivar,paid,fulfilled,Cash,,,,...,0.00,6.15,615.00,0.00,0.00,615.00,621.15,651.90,-30.75,0.00
3,#1005,1005,1005,No Customer Name,paid,fulfilled,Cash,,,,...,0.00,1.35,135.00,0.00,0.00,135.00,136.35,143.10,-6.75,0.00
4,#1006,1006,1006,Zeynel Bal,paid,fulfilled,PayPal,,,,...,0.00,4.99,499.00,0.00,0.00,499.00,503.99,528.94,-24.95,0.00
